# Build community model

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
sys.path.append('/home/UCSD/cobra_utils')
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from bacillusme.util.helper_functions import *

import copy
%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)
ecoli_files = dirname(abspath(bacillusme.__file__))

/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/__init__.py
/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/bacillusme/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f3b031ecac8>

In [3]:
for r in bsub.reactions:
    if 'ACACT' in r.id:
        print(r.id, r.reaction, r.genes)

ACACT1r 2.0 accoa_c <=> aacoa_c + coa_c frozenset({<Gene BSU24170 at 0x7f2e20a13da0>})
ACACT2r accoa_c + btcoa_c <=> 3ohcoa_c + coa_c frozenset({<Gene BSU32830 at 0x7f2e20a13d68>})
ACACT3r accoa_c + hxcoa_c <=> 3oocoa_c + coa_c frozenset({<Gene BSU32830 at 0x7f2e20a13d68>})
ACACT4r accoa_c + occoa_c <=> 3odcoa_c + coa_c frozenset({<Gene BSU32830 at 0x7f2e20a13d68>})
ACACT5r_1 accoa_c + dccoa_c <=> 3oddcoa_c + coa_c frozenset({<Gene BSU32830 at 0x7f2e20a13d68>})
ACACT6r_1 accoa_c + ddcoa_c <=> 3otdcoa_c + coa_c frozenset({<Gene BSU32830 at 0x7f2e20a13d68>})
ACACT7r accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c frozenset({<Gene BSU32830 at 0x7f2e20a13d68>})


In [4]:
with open(ecoli_files+'/me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)
with open(ecoli_files+'/me_models/solution.pickle', 'rb') as solution:
    spore = pickle.load(solution)

## Create spore model

### Change IDs (compartment _s)

In [5]:
# Metabolites
new_compartment = 's'
fix = []
for met in tqdm(spore.metabolites):
    if re.search('_[c,e]$',met.id):
        met.id = re.sub('c$',new_compartment,met.id)
    else:
        met.id = met.id+'_'+new_compartment

100%|██████████| 4211/4211 [00:00<00:00, 228471.45it/s]


In [6]:
# Reactions
new_compartment = 's'
fix = []
for r in tqdm(spore.reactions):
        r.id = r.id+'_'+new_compartment

100%|██████████| 6277/6277 [00:00<00:00, 508717.30it/s]


In [7]:
spore.repair()

In [8]:
# Create new growth key for spore
new_growth_key = 'sigma'
for r in tqdm(spore.reactions):
    lb = r.lower_bound
    ub = r.upper_bound
    new_stoichiometry = {m:s for m,s in r.metabolites.items()}
    if hasattr(lb, 'subs'):
        growth_key = list(lb.free_symbols)[0]
        r.lower_bound = lb.subs(growth_key,new_growth_key)
    if hasattr(ub, 'subs'):
        growth_key = list(ub.free_symbols)[0]
        r.upper_bound = ub.subs(growth_key,new_growth_key)
    for m,s in new_stoichiometry.items():
        if hasattr(s, 'subs'):
            growth_key = list(s.free_symbols)[0]
            new_stoichiometry[m] = s.subs(growth_key,new_growth_key)
    r.add_metabolites(new_stoichiometry,combine=False)

100%|██████████| 6277/6277 [00:38<00:00, 162.11it/s] 


In [9]:
# Eliminate exchange reactions:
reactions = [r for r in spore.reactions]
for r in tqdm(reactions):
    if re.search('^EX_',r.id):
        r.remove_from_model()

100%|██████████| 6277/6277 [00:00<00:00, 24207.68it/s]


### Create spore biomass reactions

In [10]:
bio_rxn = spore.reactions.BIOMASS_BS_10_FWD_CPLX_dummy_s

In [11]:
bio_comp = pd.read_csv('spore_composition.csv',index_col=0)

In [12]:
# Identify lipid metabolites in biomass equation
lipid_demand = {}
for m in bio_comp.index:
    if '_BS_c' in m:
        new_id = re.sub('c$','s',m)
        value = bio_comp.loc[m]['coeff']
        lipid_demand[new_id] = abs(value)
                
for met, requirement in lipid_demand.items():
    component_mass = spore.metabolites.get_by_id(met).formula_weight / 1000.
    rxn = cobrame.SummaryVariable('Demand_' + met)
    spore.add_reactions([rxn])
    rxn.add_metabolites({met: -1 * requirement,
                         'lipid_biomass_s': component_mass * requirement})
    rxn.lower_bound = spore.reactions.biomass_dilution_s.lower_bound
    rxn.upper_bound = 1000.
#     print(rxn.reaction)

### Merge

In [13]:
with open(ecoli_files+'/me_models/solution.pickle', 'rb') as solution:
    sporeme = pickle.load(solution)

In [14]:
# Mother is not growing. Reactions should then be associated to sigma
new_growth_key = 'sigma'
for r in tqdm(sporeme.reactions):
    lb = r.lower_bound
    ub = r.upper_bound
    new_stoichiometry = {m:s for m,s in r.metabolites.items()}
    if hasattr(lb, 'subs'):
        growth_key = list(lb.free_symbols)[0]
        r.lower_bound = float(lb.subs(growth_key,0.))
    if hasattr(ub, 'subs'):
        growth_key = list(ub.free_symbols)[0]
        r.upper_bound = float(ub.subs(growth_key,1000.))
    for m,s in new_stoichiometry.items():
        if hasattr(s, 'subs'):
            growth_key = list(s.free_symbols)[0]
            new_stoichiometry[m] = s.subs(growth_key,new_growth_key)
            a = 1
    r.add_metabolites(new_stoichiometry,combine=False)

100%|██████████| 6277/6277 [00:37<00:00, 166.05it/s] 


In [15]:
# sporeme.add_reactions(spore.reactions) # This approach no longer worked after python version change.

In [16]:
# Add spore reactions
for r in spore.reactions:
    new = r.__class__(r.id) # Create and identical reaction
    new.bounds = r.bounds
    sporeme.add_reaction(new)
    for k,v in r.metabolites.items():
        if not hasattr(sporeme.metabolites,k.id):
            args = k.__init__.__code__.co_varnames[1:] # Eliminate "self"
            if len(args) == 1: # No additional inputs to __init__
                met = k.__class__(k.id)
            else:
                vals = [getattr(k, dir(k)[[i.lower() for i in dir(k)].index(a)]) for a in args]
                met = k.__class__(*vals)
            sporeme.add_metabolites(met)
            met.name = k.name
            met.formula = k.formula
            met.charge = k.charge
            met.compartment = 's'
            
        new.add_metabolites({k.id:v})

In [17]:
# Connect C with S
## Change spore transport from e-s to c-s
for m in tqdm(sporeme.metabolites):
    if not isinstance(m,cobrame.Metabolite) or not re.search('_s$',m.id):
        continue
    in_transport = get_transport_reactions(sporeme,m.id,comps=['e','s'])
    out_transport = get_transport_reactions(sporeme,m.id,comps=['s','e'])
    transport_reactions = in_transport + out_transport
    for r in transport_reactions:
        old_met = sporeme.metabolites.get_by_id(re.sub('_s$','_e',m.id))
        new_met = sporeme.metabolites.get_by_id(re.sub('_s$','_c',m.id))
        coeff = r.metabolites[old_met]
        r.add_metabolites({old_met:-coeff,new_met:coeff})

100%|██████████| 8181/8181 [00:01<00:00, 4316.40it/s] 


In [18]:
sporeme.objective = 'biomass_dilution_s'
sporeme.reactions.get_by_id('BIOMASS_BS_10_FWD_CPLX_dummy_s').upper_bound = 0

### Manual fixes

As reported by Eammon, arginine is transported through permeases RocC and RocE, and ABC transporters ArtPQR.

In [19]:
# close_met_transport = ['3gmp_c','gmp_c']
# transporters = []
# for m in tqdm(close_met_transport):
#     transport = get_transport_reactions(sporeme,m,comps=['c','s']) + \
#         get_transport_reactions(sporeme,m,comps=['s','c'])
#     [transporters.append(i) for i in transport]
# transporters

In [20]:
# for t in transporters:
#     t.lower_bound = 0
#     t.upper_bound = 0

### Close specific reactions in the spore
It was shown that all but one (odhB) TCA reaction was necessary. We close the unnecessary ones.

In [21]:
# gene_dictionary = pd.read_csv('../../building_data/gene_name_dictionary.csv',index_col=1)
# eliminate_genes = ['citA','citB','citZ','icd','sucD','fumC','mdh']
# gene_dictionary.loc[eliminate_genes]['locus_id']

In [22]:
gene_dictionary = pd.read_csv('../../building_data/gene_name_dictionary.csv',index_col=1)
eliminate_genes = ['citB','citZ','icd','sucD','fumC','mdh'] # From Eammon's paper
gene_dictionary.loc[eliminate_genes]['locus_id']

name
citB    BSU18000
citZ    BSU29140
icd     BSU29130
sucD    BSU16100
fumC    BSU33040
mdh     BSU29120
Name: locus_id, dtype: object

In [23]:
# # New version
# gene_dictionary = pd.read_csv('../../building_data/gene_name_dictionary.csv',index_col=1)
# eliminate_genes = ['pckA','purL','ilvB','citZ','acsA','etfA','acoC','sigF','yxbC','lutC','argJ','hpf'] # From Eammons file of Spore proteomics
# gene_dictionary.loc[eliminate_genes]['locus_id']

In [24]:
essentiality_df = pd.read_csv('../essentiality/essentiality_results.csv',index_col=0)
eliminate_ids = set(gene_dictionary.loc[eliminate_genes]['locus_id']) & set(essentiality_df.index)
essentiality_df.loc[eliminate_ids]

,response_me,essentiality_me,response_m,essentiality_m,true_essentiality
BSU18000,-0.153926,-,-0.152083,-,0
BSU29130,-0.153936,-,-0.152083,-,0
BSU33040,-0.139421,-,-0.135523,-,0
BSU29140,-0.000028,0,0.000000,0,0
BSU29120,-0.065595,-,-0.063391,-,0
BSU16100,-0.061582,-,-0.059546,-,0


In [25]:
essentiality_df.loc[eliminate_ids][['response_me','essentiality_me','true_essentiality']]

,response_me,essentiality_me,true_essentiality
BSU18000,-0.153926,-,0
BSU29130,-0.153936,-,0
BSU33040,-0.139421,-,0
BSU29140,-0.000028,0,0
BSU29120,-0.065595,-,0
BSU16100,-0.061582,-,0


In [26]:
for gene in eliminate_ids:
    r = 'translation_' + gene +'_s'
    print(r)
    rxn = sporeme.reactions.get_by_id(r)
    rxn.upper_bound = 0
    rxn.lower_bound = 0

translation_BSU18000_s
translation_BSU29130_s
translation_BSU33040_s
translation_BSU29140_s
translation_BSU29120_s
translation_BSU16100_s


In [27]:
# Close ATP synthase as it seems it is not present in the spore according to Eammons thesis
sporeme.reactions.get_by_id('ATPS4r_FWD_CPLX000-10_s').bounds = (0,0)
# No glucose uptake from e
sporeme.reactions.get_by_id('GLCpts_FWD_BSU13890-MONOMER_s').bounds = (0,0)

In [28]:
for m in sporeme.metabolites:
    for r in m.reactions:
        r.id

### Solve

In [ ]:
solve_me_model(sporeme, max_mu = 0.12, min_mu = .05, using_soplex=False, precision = 1e-6,growth_key = 'sigma')

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 285.472195 seconds
Finished substituting S,lb,ub in 8.518893 seconds
Finished makeME_LP in 1.618053 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.06 0.06 1
Finished substituting S,lb,ub in 8.325762 seconds
Finished makeME_LP in 1.574363 seconds
Getting MINOS parameters from ME_NLP...
2 0.03 0.03 0.06 0.03 optimal
Finished substituting S,lb,ub in 8.108329 seconds
Finished makeME_LP in 1.565130 seconds
Getting MINOS parameters from ME_NLP...
3 0.045 0.045 0.06 0.045 optimal
Finished substituting S,lb,ub in 7.849010 seconds
Finished makeME_LP in 1.514287 seconds
Getting MINOS parameters from ME_NLP...
4 0.0525 0.0525 0.06 0.0525 optimal
Finished substituting S,lb,ub in 7.976000 seconds
Finished makeME_LP in 1.504055 seconds
Getting MINOS parameters from ME_NLP...
5 0.056249999999999994 0.056249999999999994 0.06 0.056249999999999994 optimal
Finished substituting S,lb,ub in 7.953759 seconds
Finishe

In [33]:
print(sporeme.solution.f)

0.021712538813805006


In [ ]:
with open("./sporeme_solution_v3.pickle", "wb") as outfile:
    pickle.dump(sporeme, outfile)

In [ ]:
with open("./sporeme_solution_v3.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [32]:
print(sporeme.solution.f)

0.021783531855335492


In [6]:
df = pd.read_csv('PEP_essential_metabolite_group_KO.csv',index_col=0)

In [9]:
for r,row in df.iterrows():
    if r not in sporeme.reactions:
        print(r)

formation_BSU31620-MONOMER
formation_BSU31660-MONOMER
formation_BSU31600-MONOMER
formation_BSU31640-MONOMER
formation_BSU31630-MONOMER
formation_BSU31650-MONOMER
formation_BSU31610-MONOMER
ETOHt3_FWD_CPLX_dummy
NAt3_1_FWD_BSU31630-MONOMER
NAt3_1_FWD_BSU31620-MONOMER
NAt3_1_FWD_BSU31600-MONOMER
NAt3_1_FWD_BSU31650-MONOMER
NAt3_1_FWD_BSU31660-MONOMER
NAt3_1_FWD_BSU31640-MONOMER
NAt3_1_FWD_BSU31610-MONOMER
RIBFLVt2_REV_CPLX_dummy
RIBFLVt2_FWD_CPLX_dummy
formation_BSU31620-MONOMER_s
formation_BSU31660-MONOMER_s
formation_BSU31600-MONOMER_s
formation_BSU31640-MONOMER_s
formation_BSU31630-MONOMER_s
formation_BSU31650-MONOMER_s
formation_BSU31610-MONOMER_s
ETOHt3_FWD_CPLX_dummy_s
NAt3_1_FWD_BSU31630-MONOMER_s
NAt3_1_FWD_BSU31620-MONOMER_s
NAt3_1_FWD_BSU31600-MONOMER_s
NAt3_1_FWD_BSU31650-MONOMER_s
NAt3_1_FWD_BSU31660-MONOMER_s
NAt3_1_FWD_BSU31640-MONOMER_s
NAt3_1_FWD_BSU31610-MONOMER_s
RIBFLVt2_REV_CPLX_dummy_s
RIBFLVt2_FWD_CPLX_dummy_s


In [37]:
for r in sporeme.reactions:
    print(r.id)

biomass_dilution
protein_biomass_to_biomass
mRNA_biomass_to_biomass
tRNA_biomass_to_biomass
rRNA_biomass_to_biomass
ncRNA_biomass_to_biomass
DNA_biomass_to_biomass
lipid_biomass_to_biomass
constituent_biomass_to_biomass
prosthetic_group_biomass_to_biomass
peptidoglycan_biomass_to_biomass
EX_2ddglcn_e
EX_2hxmp_e
EX_2pg_e
EX_2pglyc_e
EX_3amba_e
EX_3amp_e
EX_3cmp_e
EX_3gmp_e
EX_3pg_e
EX_3ump_e
EX_4abut_e
EX_5mtr_e
EX_6pgc_e
EX_Larab_e
EX_Lcyst_e
EX_abt__L_e
EX_ac_e
EX_acac_e
EX_acgam_e
EX_acmana_e
EX_acnam_e
EX_actn__R_e
EX_ade_e
EX_adn_e
EX_akg_e
EX_ala_B_e
EX_ala__D_e
EX_ala_L_Thr__L_e
EX_ala_L_asp__L_e
EX_ala__L_e
EX_ala_L_gln__L_e
EX_ala_L_glu__L_e
EX_L_alagly_e
EX_ala_L_his__L_e
EX_ala_L_leu__L_e
EX_alaala_e
EX_alltn_e
EX_amp_e
EX_amylase_e
EX_antim_e
EX_arab__D_e
EX_arab__L_e
EX_arbt_e
EX_arg__L_e
EX_argp_e
EX_arsenb_e
EX_arsna_e
EX_arsni2_e
EX_asn__L_e
EX_asp__L_e
EX_bilea_e
EX_btd_RR_e
EX_buts_e
EX_ca2_e
EX_cbl2_e
EX_cd2_e
EX_cellb_e
EX_cgly_e
EX_chitob_e
EX_chol_e
EX_chols_e
EX_c

In [28]:
len(sporeme.reactions)

12323

In [29]:
len(me.reactions) + len(spore.reactions)

12323

In [29]:
sporeme.reactions.get_by_id('transcription_TU8J2_1243_from_BSU25200-MONOMER_s').id

'transcription_TU8J2_1243_from_BSU25200-MONOMER_s'

In [72]:
with open("./sporeme_solution_v2.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [26]:
get_reactions_of_met(sporeme,'malcoa_c',only_types=['MetabolicReaction'])

( formation_CPLX000-8_s 0.0 1000.0 ) 	 protein_BSU28730_s + protein_BSU28740_s + protein_BSU28750_s --> CPLX000-8_s
( ARBabc_FWD_CPLX000-8_s 0 999999.0 ) 	 arab__L_c + atp_s + h2o_s <=> -4.27350427350427e-6*sigma CPLX000-8_s + adp_s + arab__L_s + h_s + pi_s


[<ComplexFormation formation_CPLX000-8_s at 0x7f21140ac518>,
 <MetabolicReaction ARBabc_FWD_CPLX000-8_s at 0x7f2113eb2160>]

In [10]:
len([r for r in sporeme.reactions if '_s' not in r.id[-2:]])

6054

In [11]:
len([r for r in sporeme.metabolites if '_s' in r.id[-2:]])

3973

In [15]:
len(sporeme.reactions) - 6282

6057